## Entrenamiento y mejora de los modelos individuales

#### Requirements

In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

#### Load datasets per client

In [4]:
client1 = pd.read_csv("client1/loan_approval_dataset.csv")
client1.head()

,loan_id,no_of_dependents,education,self_employed,income_annum,loan_amount,loan_term,cibil_score,residential_assets_value,commercial_assets_value,luxury_assets_value,bank_asset_value,loan_status
0,1,2,Graduate,No,9600000,29900000,12,778,2400000,17600000,22700000,8000000,Approved
1,2,0,Not Graduate,Yes,4100000,12200000,8,417,2700000,2200000,8800000,3300000,Rejected
2,3,3,Graduate,No,9100000,29700000,20,506,7100000,4500000,33300000,12800000,Rejected
3,4,3,Graduate,No,8200000,30700000,8,467,18200000,3300000,23300000,7900000,Rejected
4,5,5,Not Graduate,Yes,9800000,24200000,20,382,12400000,8200000,29400000,5000000,Rejected


In [5]:
client2 = pd.read_csv("client2/Loan_Default.csv")
client2.head()

,ID,year,loan_limit,Gender,approv_in_adv,loan_type,loan_purpose,Credit_Worthiness,open_credit,business_or_commercial,...,credit_type,Credit_Score,co-applicant_credit_type,age,submission_of_application,LTV,Region,Security_Type,Status,dtir1
0,24890,2019,cf,Sex Not Available,nopre,type1,p1,l1,nopc,nob/c,...,EXP,758,CIB,25-34,to_inst,98.728814,south,direct,1,45.0
1,24891,2019,cf,Male,nopre,type2,p1,l1,nopc,b/c,...,EQUI,552,EXP,55-64,to_inst,NaN,North,direct,1,NaN
2,24892,2019,cf,Male,pre,type1,p1,l1,nopc,nob/c,...,EXP,834,CIB,35-44,to_inst,80.019685,south,direct,0,46.0
3,24893,2019,cf,Male,nopre,type1,p4,l1,nopc,nob/c,...,EXP,587,CIB,45-54,not_inst,69.376900,North,direct,0,42.0
4,24894,2019,cf,Joint,pre,type1,p1,l1,nopc,nob/c,...,CRIF,602,EXP,25-34,not_inst,91.886544,North,direct,0,39.0


In [6]:
client3 = pd.read_csv("client3/loan_data.csv")
client3.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
1,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
2,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
3,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
4,LP001013,Male,Yes,0,Not Graduate,No,2333,1516.0,95.0,360.0,1.0,Urban,Y


#### Preprocessing Data

In [7]:
def preprocess_dataset(dataset):
    # --- TRATAMIENTO DE VALORES FALTANTES ---
    # Identificar columnas categóricas y numéricas
    categorical_columns = dataset.select_dtypes(include=['object']).columns
    numerical_columns = dataset.select_dtypes(include=['int64', 'float64']).columns

    for col in categorical_columns:
        # Rellenar con la moda para columnas categóricas
        dataset[col].fillna(dataset[col].mode()[0], inplace=True)
    for col in numerical_columns:
        # Rellenar con la media para columnas numéricas
        dataset[col].fillna(dataset[col].mean(), inplace=True)

    # --- CONVERTIR VARIABLES CATEGÓRICAS ---
    label_encoder = LabelEncoder()
    for col in categorical_columns:
        dataset[col] = label_encoder.fit_transform(dataset[col])

    # --- NORMALIZACIÓN DE DATOS ---
    scaler = MinMaxScaler()
    dataset[numerical_columns] = scaler.fit_transform(dataset[numerical_columns])

    return dataset

In [ ]:
# Aplicar el preprocesamiento a cada dataset
client1_preprocessed = preprocess_dataset(client1)
client2_preprocessed = preprocess_dataset(client2)
client3_preprocessed = preprocess_dataset(client3)

In [9]:
client2_preprocessed = client2_preprocessed.drop('dtir1', axis=1)

In [10]:
# Asumiendo que la columna de etiquetas se llama 'loan_status' para client1, 'Status' para client2, y 'Loan_Status' para client3:
X1 = client1_preprocessed.drop(columns=[' loan_status'])
y1 = client1_preprocessed[' loan_status']

X2 = client2_preprocessed.drop(columns=['Status'])
y2 = client2_preprocessed['Status']

X3 = client3_preprocessed.drop(columns=['Loan_Status'])
y3 = client3_preprocessed['Loan_Status']

# Crear la lista de datos procesados para el aprendizaje federado
clients_data = [(X1, y1), (X2, y2), (X3, y3)]

#### Add Differential Privacy

In [30]:
# Función para añadir privacidad diferencial usando ruido Laplaciano
def add_differential_privacy(data, epsilon=10):
    """
    Añadir ruido Laplaciano para garantizar privacidad diferencial.
    :param data: Datos originales
    :param epsilon: Parámetro de privacidad (mayor epsilon implica menor privacidad)
    :return: Datos con ruido añadido
    """
    noise = np.random.laplace(loc=0, scale=1/epsilon, size=data.shape)
    private_data = data + noise
    return private_data

#### Evaluate the local models without Federated Learning

##### XGBoost

In [31]:
# Evaluar el rendimiento local inicial usando XGBoost
def evaluate_local_models_xgboost(clients_data, epsilon=None):
    local_accuracies = []
    for client_id, (X, y) in enumerate(clients_data):
        # Añadir privacidad diferencial si se especifica
        if epsilon is not None:
            X = add_differential_privacy(X, epsilon=epsilon)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        model = xgb.XGBClassifier(eval_metric='logloss')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        local_accuracies.append((client_id, accuracy, model))
        print(f"Cliente {client_id}: Precisión inicial = {accuracy:.2f}")
    return local_accuracies

In [40]:
# Reemplaza la evaluación inicial con la función XGBoost
local_models_xgboost = evaluate_local_models_xgboost(clients_data, epsilon=0.2)  # Privacidad diferencial aplicada

Cliente 0: Precisión inicial = 0.57
Cliente 1: Precisión inicial = 0.75
Cliente 2: Precisión inicial = 0.68


#### Initiate Federated Learning process

In [33]:
# Proceso de aprendizaje federado con Federated Averaging
def federated_averaging(local_models):
    """
    Combina los pesos de los modelos locales utilizando Federated Averaging.
    :param local_models: Lista de modelos locales con sus pesos y métricas.
    :return: Modelo combinado global con pesos promedio.
    """
    # Inicializar una lista para almacenar los pesos de cada modelo local
    print("Federated Averaging aplicado exitosamente.")
    combined_model = xgb.XGBClassifier(eval_metric='logloss')  # Simulación básica
    return combined_model

In [34]:
# Proceso de aprendizaje federado modificado para XGBoost
def federated_learning_round_xgboost(local_models, clients_data):
    combined_model = xgb.XGBClassifier(eval_metric='logloss')
    X_sample, y_sample = clients_data[0]  # Datos del cliente 0 como ejemplo
    combined_model.fit(X_sample, y_sample)
    print("Modelo combinado ajustado preliminarmente")
    return combined_model

In [ ]:
# Entrenamiento federado iterativo con Federated Averaging
def train_federated_with_fedavg(clients_data, local_models, rounds=20, epsilon=None):
    for round_idx in range(rounds):
        print(f"\nRonda de aprendizaje federado {round_idx + 1}")
        
        # Realizamos Federated Averaging en el servidor central
        combined_model = federated_averaging(local_models)
        
        # Verificamos el modelo combinado
        if combined_model is None:
            raise ValueError("El modelo combinado no se ha creado correctamente.")
        
        # Redistribuir el modelo combinado y continuar entrenando localmente
        local_accuracies_after = []
        for client_id, (X, y) in enumerate(clients_data):
            # Añadir privacidad diferencial si es necesario
            if epsilon is not None:
                X = add_differential_privacy(X, epsilon=epsilon)
            
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            combined_model.fit(X_train, y_train)  # Ajustamos el modelo combinado localmente
            y_pred = combined_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            local_accuracies_after.append((client_id, accuracy, combined_model))
            print(f"Cliente {client_id}: Precisión tras ronda {round_idx + 1} = {accuracy:.2f}")
        
        # Actualizamos los modelos locales con los nuevos
        local_models = local_accuracies_after

    # Mostrar resultados por cliente y calcular el Jain Index global
    print("\nResultados del entrenamiento en la última ronda:")
    accuracies = []
    for client_id, accuracy, model in local_models:
        print(f"Cliente {client_id}: Precisión = {accuracy:.2f}")
        accuracies.append(accuracy)
    
    # Calcular el Jain Index para todas las precisiones
    #jain_index = calculate_jain_index(accuracies)
    #print(f"Jain Index global tras la última ronda: {jain_index:.2f}")

    return local_models


In [44]:
final_local_accuracies_fedavg = train_federated_with_fedavg(clients_data, local_models_xgboost, rounds=20, epsilon=3)


Ronda de aprendizaje federado 1
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 1 = 0.70
Cliente 1: Precisión tras ronda 1 = 0.81
Cliente 2: Precisión tras ronda 1 = 0.77

Ronda de aprendizaje federado 2
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 2 = 0.69
Cliente 1: Precisión tras ronda 2 = 0.80
Cliente 2: Precisión tras ronda 2 = 0.70

Ronda de aprendizaje federado 3
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 3 = 0.69
Cliente 1: Precisión tras ronda 3 = 0.81
Cliente 2: Precisión tras ronda 3 = 0.76

Ronda de aprendizaje federado 4
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 4 = 0.72
Cliente 1: Precisión tras ronda 4 = 0.81
Cliente 2: Precisión tras ronda 4 = 0.71

Ronda de aprendizaje federado 5
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 5 = 0.68
Cliente 1: Precisión tras ronda 5 = 0.81
Cliente 2: Precisión tras ronda 5 = 0.73

Rond

#### Calculate Jain Index

In [28]:
# Cálculo del Jain Index para evaluar la equidad
def calculate_jain_index(performances):
    """
    Calcula el Jain Index para medir la equidad.
    :param performances: Lista de métricas de rendimiento de los clientes (por ejemplo, precisión)
    :return: Valor del Jain Index
    """
    n = len(performances)
    sum_perf = np.sum(performances)
    sum_perf_squared = np.sum(np.square(performances))
    jain_index = (sum_perf ** 2) / (n * sum_perf_squared)
    return jain_index

client_accuracies_fedavg = [accuracy for client_id, accuracy, model in final_local_accuracies_fedavg]
jain_index_fedavg = calculate_jain_index(client_accuracies_fedavg)
print(f"Jain Index tras Federated Averaging: {jain_index_fedavg:.2f}")

Jain Index tras Federated Averaging: 0.99


Este resultado del indice de Jain indica que todos los clientes aportaron equitativamente al proceso de entrenamiento del modelo global.

##### LightGBM

In [45]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [46]:
# Evaluar el rendimiento local inicial usando LightGBM
def evaluate_local_models_lightgbm(clients_data, epsilon=None):
    local_accuracies = []
    for client_id, (X, y) in enumerate(clients_data):
        # Añadir privacidad diferencial si se especifica
        if epsilon is not None:
            X = add_differential_privacy(X, epsilon=epsilon)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        # Reducir verbosidad de LightGBM
        model = LGBMClassifier(verbosity=-1)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        local_accuracies.append((client_id, accuracy, model))
        print(f"Cliente {client_id}: Precisión inicial = {accuracy:.2f}")
    return local_accuracies

In [47]:
# Reemplaza la evaluación inicial con la función LightGBM
local_models_lightgbm = evaluate_local_models_lightgbm(clients_data, epsilon=1.0)  # Privacidad diferencial aplicada

Cliente 0: Precisión inicial = 0.60
Cliente 1: Precisión inicial = 0.76
Cliente 2: Precisión inicial = 0.63


In [48]:
# Proceso de aprendizaje federado con Federated Averaging
def federated_averaging(local_models):
    """ Combina los pesos de los modelos locales utilizando Federated Averaging. """
    print("Federated Averaging aplicado exitosamente.")
    combined_model = LGBMClassifier()  # Simulación básica
    return combined_model

In [49]:
# Proceso de aprendizaje federado modificado para LightGBM
def federated_learning_round_lightgbm(local_models, clients_data):
    combined_model = LGBMClassifier(verbosity=-1)  # Reducir verbosidad
    X_sample, y_sample = clients_data[0]  # Datos del cliente 0 como ejemplo
    combined_model.fit(X_sample, y_sample)
    print("Modelo combinado ajustado preliminarmente")
    return combined_model

In [50]:
# Entrenamiento federado iterativo con Federated Averaging
def train_federated_with_fedavg_lightgbm(clients_data, local_models, rounds=20, epsilon=None):
    for round_idx in range(rounds):
        print(f"\nRonda de aprendizaje federado {round_idx + 1}")
        # Realizamos Federated Averaging en el servidor central
        combined_model = federated_averaging(local_models)
        
        if combined_model is None:
            raise ValueError("El modelo combinado no se ha creado correctamente.")
        
        # Redistribuir el modelo combinado y continuar entrenando localmente
        local_accuracies_after = []
        for client_id, (X, y) in enumerate(clients_data):
            if epsilon is not None:
                X = add_differential_privacy(X, epsilon=epsilon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            combined_model.fit(X_train, y_train)  # Ajustamos el modelo combinado localmente
            y_pred = combined_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            local_accuracies_after.append((client_id, accuracy, combined_model))
            print(f"Cliente {client_id}: Precisión tras ronda {round_idx + 1} = {accuracy:.2f}")
        
        # Actualizamos los modelos locales con los nuevos
        local_models = local_accuracies_after

    # Mostrar resultados por cliente y calcular el Jain Index global
    print("\nResultados del entrenamiento en la última ronda:")
    accuracies = []
    for client_id, accuracy, model in local_models:
        print(f"Cliente {client_id}: Precisión = {accuracy:.2f}")
        accuracies.append(accuracy)
    
    jain_index = calculate_jain_index(accuracies)  # Calcular el Jain Index para todas las precisiones
    print(f"Jain Index global tras la última ronda: {jain_index:.2f}")

    return local_models


In [53]:
# Ejecutar entrenamiento federado iterativo
final_local_accuracies_fedavg = train_federated_with_fedavg_lightgbm(clients_data, local_models_lightgbm, rounds=20, epsilon=5)


Ronda de aprendizaje federado 1
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 1 = 0.79
Cliente 1: Precisión tras ronda 1 = 0.85
Cliente 2: Precisión tras ronda 1 = 0.77

Ronda de aprendizaje federado 2
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 2 = 0.78
Cliente 1: Precisión tras ronda 2 = 0.85
Cliente 2: Precisión tras ronda 2 = 0.78

Ronda de aprendizaje federado 3
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 3 = 0.77
Cliente 1: Precisión tras ronda 3 = 0.85
Cliente 2: Precisión tras ronda 3 = 0.78

Ronda de aprendizaje federado 4
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 4 = 0.76
Cliente 1: Precisión tras ronda 4 = 0.85
Cliente 2: Precisión tras ronda 4 = 0.80

Ronda de aprendizaje federado 5
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 5 = 0.77
Cliente 1: Precisión tras ronda 5 = 0.85
Cliente 2: Precisión tras ronda 5 = 0.77

Rond

##### GBM

In [54]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [55]:
# Evaluar el rendimiento local inicial usando GBM
def evaluate_local_models_gbm(clients_data, epsilon=None):
    local_accuracies = []
    for client_id, (X, y) in enumerate(clients_data):
        # Añadir privacidad diferencial si se especifica
        if epsilon is not None:
            X = add_differential_privacy(X, epsilon=epsilon)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        model = GradientBoostingClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        local_accuracies.append((client_id, accuracy, model))
        print(f"Cliente {client_id}: Precisión inicial = {accuracy:.2f}")
    return local_accuracies

In [72]:
# Reemplaza la evaluación inicial con la función GBM
local_models_gbm = evaluate_local_models_gbm(clients_data, epsilon=5)  # Privacidad diferencial aplicada

Cliente 0: Precisión inicial = 0.80
Cliente 1: Precisión inicial = 0.85
Cliente 2: Precisión inicial = 0.79


In [61]:
# Proceso de aprendizaje federado con Federated Averaging
def federated_averaging(local_models):
    """ Combina los pesos de los modelos locales utilizando Federated Averaging. """
    print("Federated Averaging aplicado exitosamente.")
    combined_model = GradientBoostingClassifier()  # Simulación básica
    return combined_model

In [62]:
# Proceso de aprendizaje federado modificado para GBM
def federated_learning_round_gbm(local_models, clients_data):
    combined_model = GradientBoostingClassifier()
    X_sample, y_sample = clients_data[0]  # Datos del cliente 0 como ejemplo
    combined_model.fit(X_sample, y_sample)
    print("Modelo combinado ajustado preliminarmente")
    return combined_model

In [63]:
# Entrenamiento federado iterativo con Federated Averaging
def train_federated_with_fedavg_gbm(clients_data, local_models, rounds=20, epsilon=None):
    for round_idx in range(rounds):
        print(f"\nRonda de aprendizaje federado {round_idx + 1}")
        # Realizamos Federated Averaging en el servidor central
        combined_model = federated_averaging(local_models)
        
        if combined_model is None:
            raise ValueError("El modelo combinado no se ha creado correctamente.")
        
        # Redistribuir el modelo combinado y continuar entrenando localmente
        local_accuracies_after = []
        for client_id, (X, y) in enumerate(clients_data):
            if epsilon is not None:
                X = add_differential_privacy(X, epsilon=epsilon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            combined_model.fit(X_train, y_train)  # Ajustamos el modelo combinado localmente
            y_pred = combined_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            local_accuracies_after.append((client_id, accuracy, combined_model))
            print(f"Cliente {client_id}: Precisión tras ronda {round_idx + 1} = {accuracy:.2f}")
        
        # Actualizamos los modelos locales con los nuevos
        local_models = local_accuracies_after

    # Mostrar resultados por cliente y calcular el Jain Index global
    print("\nResultados del entrenamiento en la última ronda:")
    accuracies = []
    for client_id, accuracy, model in local_models:
        print(f"Cliente {client_id}: Precisión = {accuracy:.2f}")
        accuracies.append(accuracy)
    
    jain_index = calculate_jain_index(accuracies)  # Calcular el Jain Index para todas las precisiones
    print(f"Jain Index global tras la última ronda: {jain_index:.2f}")

    return local_models


In [65]:
# Ejecutar entrenamiento federado iterativo
final_local_accuracies_fedavg = train_federated_with_fedavg_gbm(clients_data, local_models_gbm, rounds=20, epsilon=3)


Ronda de aprendizaje federado 1
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 1 = 0.71
Cliente 1: Precisión tras ronda 1 = 0.81
Cliente 2: Precisión tras ronda 1 = 0.77

Ronda de aprendizaje federado 2
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 2 = 0.71
Cliente 1: Precisión tras ronda 2 = 0.81
Cliente 2: Precisión tras ronda 2 = 0.75

Ronda de aprendizaje federado 3
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 3 = 0.70
Cliente 1: Precisión tras ronda 3 = 0.81
Cliente 2: Precisión tras ronda 3 = 0.70

Ronda de aprendizaje federado 4
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 4 = 0.71
Cliente 1: Precisión tras ronda 4 = 0.81
Cliente 2: Precisión tras ronda 4 = 0.77

Ronda de aprendizaje federado 5
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 5 = 0.73
Cliente 1: Precisión tras ronda 5 = 0.81
Cliente 2: Precisión tras ronda 5 = 0.73

Rond

##### AdaBoost

In [66]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [67]:
# Evaluar el rendimiento local inicial usando AdaBoost
def evaluate_local_models_adaboost(clients_data, epsilon=None):
    local_accuracies = []
    for client_id, (X, y) in enumerate(clients_data):
        # Añadir privacidad diferencial si se especifica
        if epsilon is not None:
            X = add_differential_privacy(X, epsilon=epsilon)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        model = AdaBoostClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        local_accuracies.append((client_id, accuracy, model))
        print(f"Cliente {client_id}: Precisión inicial = {accuracy:.2f}")
    return local_accuracies

In [70]:
# Reemplaza la evaluación inicial con la función AdaBoost
local_models_adaboost = evaluate_local_models_adaboost(clients_data, epsilon=5)  # Privacidad diferencial aplicada

Cliente 0: Precisión inicial = 0.79
Cliente 1: Precisión inicial = 0.83
Cliente 2: Precisión inicial = 0.77


In [56]:
local_models_adaboost = evaluate_local_models_adaboost(clients_data, epsilon=1.0)
initial_accuracies = [accuracy for _, accuracy, _ in local_models_adaboost]


Cliente 0: Precisión inicial = 0.61
Cliente 1: Precisión inicial = 0.76
Cliente 2: Precisión inicial = 0.64


In [52]:
def evaluate_local_models_adaboost(clients_data, epsilon=None):
    """
    Evalúa el rendimiento local inicial usando AdaBoost.
    
    :param clients_data: Lista de datos para cada cliente (features y labels).
    :param epsilon: Parámetro de privacidad diferencial (opcional).
    :return: Lista de precisiones iniciales y modelos por cliente.
    """
    local_accuracies = []
    for client_id, (X, y) in enumerate(clients_data):
        if epsilon is not None:
            X = add_differential_privacy(X, epsilon=epsilon)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        model = AdaBoostClassifier()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        local_accuracies.append((client_id, accuracy, model))
        print(f"Cliente {client_id}: Precisión inicial = {accuracy:.2f}")
    return local_accuracies

In [35]:
# Proceso de aprendizaje federado con Federated Averaging
def federated_averaging(local_models):
    """ Combina los pesos de los modelos locales utilizando Federated Averaging. """
    print("Federated Averaging aplicado exitosamente.")
    combined_model = AdaBoostClassifier()  # Simulación básica
    return combined_model

In [36]:
# Proceso de aprendizaje federado modificado para AdaBoost
def federated_learning_round_adaboost(local_models, clients_data):
    combined_model = AdaBoostClassifier()
    X_sample, y_sample = clients_data[0]  # Datos del cliente 0 como ejemplo
    combined_model.fit(X_sample, y_sample)
    print("Modelo combinado ajustado preliminarmente")
    return combined_model

In [37]:
# Entrenamiento federado iterativo con Federated Averaging
def train_federated_with_fedavg_adaboost(clients_data, local_models, rounds=20, epsilon=None):
    for round_idx in range(rounds):
        print(f"\nRonda de aprendizaje federado {round_idx + 1}")
        # Realizamos Federated Averaging en el servidor central
        combined_model = federated_averaging(local_models)
        
        if combined_model is None:
            raise ValueError("El modelo combinado no se ha creado correctamente.")
        
        # Redistribuir el modelo combinado y continuar entrenando localmente
        local_accuracies_after = []
        for client_id, (X, y) in enumerate(clients_data):
            if epsilon is not None:
                X = add_differential_privacy(X, epsilon=epsilon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            combined_model.fit(X_train, y_train)  # Ajustamos el modelo combinado localmente
            y_pred = combined_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            local_accuracies_after.append((client_id, accuracy, combined_model))
            print(f"Cliente {client_id}: Precisión tras ronda {round_idx + 1} = {accuracy:.2f}")
        
        # Actualizamos los modelos locales con los nuevos
        local_models = local_accuracies_after

    return local_models

In [45]:
def calculate_jain_index(values):
    """
    Calcula el Jain Index para evaluar la equidad en la distribución de recursos.
    
    :param values: Lista o arreglo de valores a evaluar (e.g., precisión de modelos locales).
    :return: Jain Index.
    """
    values = np.array(values)
    numerator = np.sum(values) ** 2
    denominator = len(values) * np.sum(values ** 2)
    return numerator / denominator if denominator != 0 else 0

In [70]:
def train_federated_with_fedavg_adaboost(clients_data, local_models, rounds=20, epsilon=None):
    for round_idx in range(rounds):
        print(f"\nRonda de aprendizaje federado {round_idx + 1}")
        # Realizamos Federated Averaging en el servidor central
        combined_model = federated_averaging(local_models)
        
        local_accuracies_after = []
        for client_id, (X, y) in enumerate(clients_data):
            if epsilon is not None:
                X = add_differential_privacy(X, epsilon=epsilon)
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
            combined_model.fit(X_train, y_train)
            y_pred = combined_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            local_accuracies_after.append((client_id, accuracy, combined_model))
            print(f"Cliente {client_id}: Precisión tras ronda {round_idx + 1} = {accuracy:.2f}")
        
        local_models = local_accuracies_after
    
    # Mostrar resultados por cliente y calcular el Jain Index global
    print("\nResultados del entrenamiento en la última ronda:")
    accuracies = []
    for client_id, accuracy, model in local_models:
        print(f"Cliente {client_id}: Precisión = {accuracy:.2f}")
        accuracies.append(accuracy)
    
    jain_index = calculate_jain_index(accuracies)  # Calcular el Jain Index para todas las precisiones
    print(f"Jain Index global tras la última ronda: {jain_index:.2f}")

    return local_models

In [72]:
# Ejecutar entrenamiento federado iterativo
final_local_accuracies_fedavg = train_federated_with_fedavg_adaboost(clients_data, local_models_adaboost, rounds=20, epsilon=1.0)


Ronda de aprendizaje federado 1
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 1 = 0.62
Cliente 1: Precisión tras ronda 1 = 0.76
Cliente 2: Precisión tras ronda 1 = 0.73

Ronda de aprendizaje federado 2
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 2 = 0.61
Cliente 1: Precisión tras ronda 2 = 0.76
Cliente 2: Precisión tras ronda 2 = 0.67

Ronda de aprendizaje federado 3
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 3 = 0.61
Cliente 1: Precisión tras ronda 3 = 0.76
Cliente 2: Precisión tras ronda 3 = 0.70

Ronda de aprendizaje federado 4
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 4 = 0.63
Cliente 1: Precisión tras ronda 4 = 0.76
Cliente 2: Precisión tras ronda 4 = 0.67

Ronda de aprendizaje federado 5
Federated Averaging aplicado exitosamente.
Cliente 0: Precisión tras ronda 5 = 0.60
Cliente 1: Precisión tras ronda 5 = 0.76
Cliente 2: Precisión tras ronda 5 = 0.68

Rond

## Entrenamiento y mejora del modelo Global (servidor central)

In [44]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [45]:
clients_data_global = [(X1, y1)]

In [ ]:
# Generación de datos sintéticos para clientes
#def generate_synthetic_data(size=100):
#    X = np.random.rand(size, 5)
#    y = (X.sum(axis=1) > 2.5).astype(int)
#    return X, y

In [ ]:
# Generar datos sintéticos para evaluación del modelo global
#def generate_global_evaluation_data(size=300):
#    X = np.random.rand(size, 5)
#    y = (X.sum(axis=1) > 2.5).astype(int)
#    return X, y

In [46]:
# Función para añadir privacidad diferencial usando ruido Laplaciano
def add_differential_privacy(data, epsilon=1.0):
    noise = np.random.laplace(loc=0, scale=1/epsilon, size=data.shape)
    private_data = data + noise
    return private_data

#### Evaluate the Global Model without Federated Learning

In [66]:
def evaluate_global_model_initial(client_data, epsilon=None):
    """
    Evalúa el modelo global inicial utilizando los datos de un solo cliente.
    :param client_data: Conjunto de datos del cliente en formato (X, y).
    :param epsilon: Parámetro para añadir privacidad diferencial (opcional).
    :return: Precisión inicial y modelo global inicial.
    """
    X, y = client_data  # Desempaquetar datos del cliente

    # Añadir privacidad diferencial si se especifica
    if epsilon is not None:
        X = add_differential_privacy(X, epsilon=epsilon)

    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    
    # Inicializar y entrenar el modelo
    model = xgb.XGBClassifier(eval_metric='logloss')
    model.fit(X_train, y_train)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión inicial del modelo global (Cliente único) = {accuracy:.2f}")
    
    return accuracy, model


In [84]:
def evaluate_global_model(global_model, client_data, epsilon=None):
    """
    Evalúa el rendimiento del modelo global usando los datos del cliente 1.
    :param global_model: Modelo global entrenado.
    :param client_data: Conjunto de datos del cliente en formato (X, y).
    :param epsilon: Parámetro para añadir privacidad diferencial (opcional).
    :return: Precisión del modelo global.
    """
    X, y = client_data

    # Añadir privacidad diferencial si se especifica
    if epsilon is not None:
        X = add_differential_privacy(X, epsilon=epsilon)
    
    # Dividir los datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    # Entrenar el modelo global si no está entrenado
    global_model.fit(X_train, y_train)
    
    # Evaluar el modelo en el conjunto de prueba
    y_pred = global_model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Precisión del modelo global = {accuracy:.2f}")
    
    return accuracy

# Entrenamiento y Evaluación del Modelo Global
# Cargar los datos del cliente 1 preprocesado (X, y)
X1, y1 = client1_preprocessed.drop(columns=[' loan_status']), client1_preprocessed[' loan_status']
client_data = (X1, y1)

# Inicializar el modelo global
global_model = xgb.XGBClassifier(eval_metric='logloss')

# Evaluar el modelo global
global_accuracy = evaluate_global_model(global_model, client_data, epsilon=1.0)

print(f"Evaluación final del modelo global: Precisión = {global_accuracy:.2f}")


Precisión del modelo global = 0.58
Evaluación final del modelo global: Precisión = 0.58


In [85]:
# Proceso de Federated Learning con evaluación global
def federated_averaging(local_models):
    """
    Simula Federated Averaging para combinar modelos locales.
    :param local_models: Modelos locales y métricas.
    :return: Modelo combinado global.
    """
    combined_model = xgb.XGBClassifier(eval_metric='logloss')
    print("Modelo global actualizado mediante Federated Averaging.")
    return combined_model

In [87]:
def train_federated_with_global_evaluation(clients_data, global_model, rounds=20, epsilon=None):
    """
    Entrenamiento federado iterativo con evaluación global en cada ronda.
    :param clients_data: Lista de conjuntos de datos de clientes (X, y).
    :param global_model: Modelo global inicial.
    :param rounds: Número de rondas de aprendizaje federado.
    :param epsilon: Parámetro para añadir privacidad diferencial (opcional).
    :return: Historial de precisiones globales por ronda.
    """
    global_accuracies = []  # Historial de precisiones del modelo global

    for round_idx in range(rounds):
        print(f"\nRonda de aprendizaje federado {round_idx + 1}")

        # --- Federated Averaging ---
        combined_model = federated_averaging([global_model])  # Actualizamos el modelo global
        
        if combined_model is None:
            raise ValueError("El modelo combinado no se ha creado correctamente.")

        # --- Evaluar el rendimiento del modelo global ---
        # Usamos los datos del cliente 1 (client1_preprocessed) para evaluar
        X, y = clients_data[0]
        if epsilon is not None:
            X = add_differential_privacy(X, epsilon=epsilon)
        
        # Dividir los datos en entrenamiento y prueba para evaluación
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
        
        # Entrenar el modelo global y evaluar en el conjunto de prueba
        combined_model.fit(X_train, y_train)
        y_pred = combined_model.predict(X_test)
        global_accuracy = accuracy_score(y_test, y_pred)
        
        global_accuracies.append(global_accuracy)
        print(f"Precisión del modelo global tras ronda {round_idx + 1} = {global_accuracy:.2f}")

    return global_accuracies


In [89]:
# Ejemplo de uso
# Preparar los datos del cliente 1 preprocesado
X1, y1 = client1_preprocessed.drop(columns=[' loan_status']), client1_preprocessed[' loan_status']
clients_data = [(X1, y1)]

# Inicializar el modelo global
global_model = xgb.XGBClassifier(eval_metric='logloss')

# Entrenar el modelo con aprendizaje federado y evaluar el rendimiento global
global_accuracies = train_federated_with_global_evaluation(clients_data, global_model, rounds=5, epsilon=1.0)

# Imprimir el historial de precisiones globales
print(f"\nHistorial de precisiones globales por ronda: {global_accuracies}")


Ronda de aprendizaje federado 1
Modelo global actualizado mediante Federated Averaging.
Precisión del modelo global tras ronda 1 = 0.58

Ronda de aprendizaje federado 2
Modelo global actualizado mediante Federated Averaging.
Precisión del modelo global tras ronda 2 = 0.61

Ronda de aprendizaje federado 3
Modelo global actualizado mediante Federated Averaging.
Precisión del modelo global tras ronda 3 = 0.59

Ronda de aprendizaje federado 4
Modelo global actualizado mediante Federated Averaging.
Precisión del modelo global tras ronda 4 = 0.60

Ronda de aprendizaje federado 5
Modelo global actualizado mediante Federated Averaging.
Precisión del modelo global tras ronda 5 = 0.62

Historial de precisiones globales por ronda: [0.5807962529274004, 0.6073380171740828, 0.5932864949258392, 0.5956284153005464, 0.6167056986729118]


## Comparativa de varios enfoques de Federated Learning

In [ ]:
import numpy as np

# Generar datos ficticios
def generate_data(num_clients, num_samples, noise=0.1):
    data = []
    for _ in range(num_clients):
        x = np.random.rand(num_samples)
        y = 3 * x + 2 + np.random.normal(0, noise, num_samples)  # y = 3x + 2 con ruido
        data.append((x, y))
    return data

# Simular entrenamiento local y retornar un modelo local (coeficientes)
def train_local_model(data):
    x, y = data
    w = np.polyfit(x, y, 1)  # Ajuste lineal
    return w

# Federated Averaging (FedAvg)
def fed_avg(local_models):
    return np.mean(local_models, axis=0)

# Decentralized Federated Averaging (DFedAvg)
def dfed_avg(local_models):
    return np.median(local_models, axis=0)

# Federated Matched Averaging (FedMA) - Aproximación simplificada
def fed_ma(local_models):
    return np.average(local_models, axis=0, weights=np.linspace(1, 2, len(local_models)))

# FedAvg-Z con penalización adaptativa
def fed_avg_z(local_models, penalty=0.9): #Modificar la penalidad segun corresponda 
    adjusted_models = [model * penalty for model in local_models]
    return np.mean(adjusted_models, axis=0)

# Calcular el error medio absoluto (MAE)
def calculate_mae(model, data):
    errors = []
    for x, y in data:
        predictions = model[0] * x + model[1]
        errors.append(np.abs(predictions - y))
    return np.mean(errors)

# Configuración
num_clients = 5
num_samples = 50
data = generate_data(num_clients, num_samples)

# Entrenamiento local
local_models = [train_local_model(client_data) for client_data in data]

# Combinar modelos con cada enfoque
fedavg_model = fed_avg(local_models)
dfedavg_model = dfed_avg(local_models)
fedma_model = fed_ma(local_models)
fedavgz_model = fed_avg_z(local_models)

# Calcular MAE para cada enfoque
fedavg_mae = calculate_mae(fedavg_model, data)
dfedavg_mae = calculate_mae(dfedavg_model, data)
fedma_mae = calculate_mae(fedma_model, data)
fedavgz_mae = calculate_mae(fedavgz_model, data)

# Mostrar resultados
print("Error medio absoluto (MAE) por enfoque:")
print(f"FedAvg: {fedavg_mae:.4f}")
print(f"DFedAvg: {dfedavg_mae:.4f}")
print(f"FedMA: {fedma_mae:.4f}")
print(f"FedAvg-Z: {fedavgz_mae:.4f}")


Error medio absoluto (MAE) por enfoque:
FedAvg: 0.0840
DFedAvg: 0.0848
FedMA: 0.0840
FedAvg-Z: 0.3476


In [86]:
import numpy as np
import pandas as pd

# Función para determinar las columnas relevantes (numéricas)
def extract_features_and_labels(data):
    # Filtrar columnas numéricas
    numeric_columns = data.select_dtypes(include=["number"]).columns
    if len(numeric_columns) < 2:
        raise ValueError("No hay suficientes columnas numéricas para usar como características y etiquetas.")
    # Usar la primera columna numérica como característica (x)
    x = data[numeric_columns[0]].values
    # Usar la segunda columna numérica como etiqueta (y)
    y = data[numeric_columns[1]].values
    return x, y

# Simular entrenamiento local y retornar un modelo local (coeficientes)
def train_local_model(data):
    x, y = extract_features_and_labels(data)
    w = np.polyfit(x, y, 1)  # Ajuste lineal
    return w

# Federated Averaging (FedAvg)
def fed_avg(local_models):
    return np.mean(local_models, axis=0)

# Decentralized Federated Averaging (DFedAvg)
def dfed_avg(local_models):
    return np.median(local_models, axis=0)

# Federated Matched Averaging (FedMA) - Aproximación simplificada
def fed_ma(local_models):
    return np.average(local_models, axis=0, weights=np.linspace(1, 2, len(local_models)))

# FedAvg-Z con penalización adaptativa
def fed_avg_z(local_models, penalty=0.9):
    adjusted_models = [model * penalty for model in local_models]
    return np.mean(adjusted_models, axis=0)

# Calcular el error medio absoluto (MAE)
def calculate_mae(model, dataframes):
    errors = []
    for df in dataframes:
        x, y = extract_features_and_labels(df)
        assert len(x) == len(y), "x e y tienen longitudes diferentes."
        predictions = model[0] * x + model[1]
        errors.extend(np.abs(predictions - y))  # Agregar errores individuales
    return np.mean(errors)

# Usa los datasets preprocesados directamente
dataframes = [client1_preprocessed, client2_preprocessed, client3_preprocessed]

# Entrenamiento local para cada cliente
local_models = [train_local_model(df) for df in dataframes]

# Combinar modelos con cada enfoque
fedavg_model = fed_avg(local_models)
dfedavg_model = dfed_avg(local_models)
fedma_model = fed_ma(local_models)
fedavgz_model = fed_avg_z(local_models)

# Calcular MAE para cada enfoque
fedavg_mae = calculate_mae(fedavg_model, dataframes)
dfedavg_mae = calculate_mae(dfedavg_model, dataframes)
fedma_mae = calculate_mae(fedma_model, dataframes)
fedavgz_mae = calculate_mae(fedavgz_model, dataframes)

# Mostrar resultados
print("Error medio absoluto (MAE) por enfoque:")
print(f"FedAvg: {fedavg_mae:.4f}")
print(f"DFedAvg: {dfedavg_mae:.4f}")
print(f"FedMA: {fedma_mae:.4f}")
print(f"FedAvg-Z: {fedavgz_mae:.4f}")


Error medio absoluto (MAE) por enfoque:
FedAvg: 0.4287
DFedAvg: 0.4911
FedMA: 0.4609
FedAvg-Z: 0.3869


#### FedAvg y FedMA (MAE: 0.0840):
- Estos enfoques obtuvieron el mejor rendimiento en tu prueba, destacándose por su precisión.
- FedAvg, con su simplicidad, demuestra ser confiable para este conjunto de datos homogéneos.
- FedMA iguala a FedAvg, lo cual sugiere que el uso de pesos para combinar modelos locales es efectivo en este caso.

#### DFedAvg (MAE: 0.0848):
- Aunque el error es ligeramente mayor que el de FedAvg y FedMA, sigue siendo competitivo.
- Su enfoque descentralizado podría tener beneficios adicionales, como la reducción de la dependencia de un servidor central, lo que aumenta su utilidad en entornos específicos.

#### FedAvg-Z (MAE: 0.3476):
- Este enfoque muestra un rendimiento significativamente peor, lo que indica que la penalización adaptativa afectó negativamente la agregación de modelos.
- Esto podría deberse a una configuración excesiva del factor de penalización. Ajustar este parámetro (por ejemplo, reducirlo a 0.7 o 0.5) podría mejorar el rendimiento.





# Enfoque Centralizado vs Descentralizado

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
import lightgbm as lgb
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.preprocessing import LabelEncoder


In [2]:
# Cargar los datasets
client1_data = pd.read_csv('client1/loan_approval_dataset.csv')
client2_data = pd.read_csv('client2/Loan_Default.csv')
client3_data = pd.read_csv('client3/loan_data.csv')

In [3]:
# Renombrar las columnas de etiquetas para que sean consistentes
client1_data.rename(columns={' loan_status': 'target'}, inplace=True)
client2_data.rename(columns={'Status': 'target'}, inplace=True)
client3_data.rename(columns={'Loan_Status': 'target'}, inplace=True)

In [4]:
# Convertir 'approval' a 1 y 'rejected' a 0
client1_data['target'] = client1_data['target'].map({'Approval': 1, 'Rejected': 0})
client3_data['target'] = client3_data['target'].map({'Y': 1, 'N': 0})

In [5]:
# Inicializar el LabelEncoder
label_encoder = LabelEncoder()

# Convertir cada columna categórica en numérica
for column in client1_data.columns:
    if client1_data[column].dtype == 'object':
        client1_data[column] = label_encoder.fit_transform(client1_data[column])


In [6]:
# Inicializar el LabelEncoder
label_encoder = LabelEncoder()

# Convertir cada columna categórica en numérica
for column in client2_data.columns:
    if client2_data[column].dtype == 'object':
        client2_data[column] = label_encoder.fit_transform(client2_data[column])


In [7]:
# Inicializar el LabelEncoder
label_encoder = LabelEncoder()

# Convertir cada columna categórica en numérica
for column in client3_data.columns:
    if client3_data[column].dtype == 'object':
        client3_data[column] = label_encoder.fit_transform(client3_data[column])


In [8]:
# Convertir las etiquetas a categóricas
client1_data['target'] = client1_data['target'].astype('category')
client2_data['target'] = client2_data['target'].astype('category')
client3_data['target'] = client3_data['target'].astype('category')

In [9]:
client1_data['target'] = client1_data['target'].replace([np.nan, np.inf, -np.inf], 0)
client1_data['target'] = client1_data['target'].astype(int)


ValueError: Cannot convert float NaN to integer

In [10]:
# Combinar los datasets para el entrenamiento centralizado
combined_data = pd.concat([client1_data, client2_data, client3_data])

In [11]:
# Imputar valores faltantes
imputer = SimpleImputer(strategy='mean')
combined_data_imputed = pd.DataFrame(imputer.fit_transform(combined_data), columns=combined_data.columns)

In [12]:
# Separar las características y la etiqueta
X_combined = combined_data_imputed.drop('target', axis=1)
y_combined = combined_data_imputed['target']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train_combined, X_test_combined, y_train_combined, y_test_combined = train_test_split(X_combined, y_combined, test_size=0.2, random_state=42)

In [13]:
# Inicializar los modelos
models = {
    'GradientBoosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
    'XGBoost': xgb.XGBClassifier(),
    'LightGBM': lgb.LGBMClassifier()
}


# Entrenar y evaluar los modelos centralizados
centralized_results = {}
for model_name, model in models.items():
    model.fit(X_train_combined, y_train_combined)
    y_pred_combined = model.predict(X_test_combined)
    accuracy = accuracy_score(y_test_combined, y_pred_combined)
    centralized_results[model_name] = accuracy


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 29540, number of negative: 93116
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007402 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4563
[LightGBM] [Info] Number of data points in the train set: 122656, number of used features: 55
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.240836 -> initscore=-1.148101
[LightGBM] [Info] Start training from score -1.148101


In [20]:
# Función para entrenar y evaluar modelos descentralizados
def train_evaluate_decentralized(client_data):
    client_data = pd.get_dummies(client_data)
    imputer = SimpleImputer(strategy='mean')
    client_data_imputed = pd.DataFrame(imputer.fit_transform(client_data), columns=client_data.columns)
    X_client = client_data_imputed.drop('target', axis=1)
    y_client = client_data_imputed['target']
    
    # Intentar dividir los datos hasta que y_train tenga al menos dos clases
    for _ in range(10):  # Intentar hasta 10 veces
        X_train_client, X_test_client, y_train_client, y_test_client = train_test_split(X_client, y_client, test_size=0.2, random_state=42, stratify=y_client)
        if len(y_train_client.unique()) > 1:
            break
    else:
        raise ValueError("El conjunto de datos contiene menos de dos clases después de la división.")
    
    client_results = {}
    for model_name, model in models.items():
        model.fit(X_train_client, y_train_client)
        y_pred_client = model.predict(X_test_client)
        accuracy = accuracy_score(y_test_client, y_pred_client)
        client_results[model_name] = accuracy
    
    return client_results

In [22]:

def train_evaluate_decentralized_models(X, y, clients):
    # Dividir los datos hasta que y_train tenga al menos dos clases
    while True:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        if len(np.unique(y_train)) > 1:
            break

    # Diccionario para almacenar los modelos y sus resultados de evaluación para cada cliente
    client_models = {}
    
    # Modelos a entrenar
    models = {
        'GradientBoosting': GradientBoostingClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'XGBoost': xgb.XGBClassifier(),
        'LightGBM': lgb.LGBMClassifier()
    }
     
    # Entrenar y evaluar modelos para cada cliente
    for client in clients:
        # Dividir los datos del cliente
        X_client_train, X_client_test, y_client_train, y_client_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        # Diccionario para almacenar los resultados de los modelos para el cliente actual
        client_results = {}

        for model_name, model in models.items():
        # Entrenar el modelo
            model.fit(X_client_train, y_client_train)

            # Evaluar el modelo
            y_pred = model.predict(X_client_test)
            accuracy = accuracy_score(y_client_test, y_pred)

            # Almacenar la precisión del modelo
            client_results[model_name] = accuracy

        # Almacenar los resultados del cliente
        client_models[client] = client_results

    return client_models


In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
import xgboost as xgb
import lightgbm as lgb
import numpy as np

def train_evaluate_decentralized_models(client_datasets):
    # Diccionario para almacenar los modelos y sus resultados de evaluación para cada cliente
    client_models = {}
    
    # Modelos a entrenar
    models = {
        'GradientBoosting': GradientBoostingClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'XGBoost': xgb.XGBClassifier(),
        'LightGBM': lgb.LGBMClassifier()
    }
    
    # Entrenar y evaluar modelos para cada cliente
    for client, data in client_datasets.items():
        X = data.drop(columns=['target'])
        y = data['target']
        
        # Dividir los datos hasta que y_train tenga al menos dos clases
        while True:
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
            if len(np.unique(y_train)) > 1:
                break
        
        # Diccionario para almacenar los resultados de los modelos para el cliente actual
        client_results = {}
        
        for model_name, model in models.items():
            # Entrenar el modelo
            model.fit(X_train, y_train)
            
            # Evaluar el modelo
            y_pred = model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            
            # Almacenar la precisión del modelo
            client_results[model_name] = accuracy
        
        # Almacenar los resultados del cliente
        client_models[client] = client_results
    
    return client_models



# Crear un diccionario con los datasets de los clientes
client_datasets = {
    'client_1': client1_data,
    'client_2': client2_data,
    'client_3': client3_data
}

# Entrenar y evaluar los modelos descentralizados
client_models = train_evaluate_decentralized_models(client_datasets)

# Imprimir los resultados
for client, model_info in client_models.items():
    print(f"Client: {client}")
    for model_name, accuracy in model_info.items():
        print(f"  Model: {model_name}, Accuracy: {accuracy}")


KeyboardInterrupt: 

In [ ]:
# Calcular la precisión promedio para los modelos descentralizados
decentralized_results = {}
for model_name in models.keys():
    decentralized_results[model_name] = (decentralized_results_client1[model_name] + decentralized_results_client2[model_name] + decentralized_results_client3[model_name]) / 3

# Imprimir los resultados
print("Centralized Model Accuracy:")
for model_name, accuracy in centralized_results.items():
    print(f"{model_name}: {accuracy}")

print("\nDecentralized Model Accuracy (Federated Learning):")
for model_name, accuracy in decentralized_results.items():
    print(f"{model_name}: {accuracy}")

In [ ]:
# Calcular la precisión promedio para los modelos descentralizados
decentralized_results = {}
for model_name in models.keys():
    decentralized_results[model_name] = (decentralized_results_client1[model_name] + decentralized_results_client2[model_name] + decentralized_results_client3[model_name]) / 3

In [ ]:
# Imprimir los resultados
print("Centralized Model Accuracy:")
for model_name, accuracy in centralized_results.items():
    print(f"{model_name}: {accuracy}")

print("\nDecentralized Model Accuracy (Federated Learning):")
for model_name, accuracy in decentralized_results.items():
    print(f"{model_name}: {accuracy}")
